In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, MultiTaskElasticNetCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from joblib import dump, load
import hdbscan
import shap
import warnings
import re

# Suppress warnings
warnings.simplefilter('ignore')

In [ ]:
def model_test(ytest, ypred):
  '''
  runs MSE, R2, RMSE, and MAE
  '''
  mse = mean_squared_error(ytest, ypred)
  r2 = r2_score(ytest, ypred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(ytest, ypred)

  return mse, r2, rmse, mae

KNN Imputation

In [ ]:
#cleaning the raw data
raw_data = pd.read_csv('~/Downloads/sheet1.csv')
df = raw_data.dropna(how='all')[:-2].drop(columns=['Country Code','Time Code']).replace('..', np.nan)
feature_matrix = df.iloc[:,2:]
for col in feature_matrix.columns:
    feature_matrix[col] = pd.to_numeric(feature_matrix[col], errors='coerce')
cleaned_df = pd.concat([df.iloc[:,0:2], feature_matrix], axis=1).reset_index().iloc[:, 1:]

#imputing using knn
scaler = StandardScaler()
data_scaled_df = pd.DataFrame(scaler.fit_transform(feature_matrix), columns=feature_matrix.columns, index=feature_matrix.index)
imputer = KNNImputer(n_neighbors=2)
data_imputed = imputer.fit_transform(data_scaled_df)
imputed_df = pd.DataFrame(data_imputed, columns=feature_matrix.columns, index=feature_matrix.index)

#inversing the scaling so we keep original information.
scaled_imputed_df = pd.DataFrame(scaler.inverse_transform(imputed_df), columns=feature_matrix.columns, index=feature_matrix.index)
scaled_imputed_final_df = pd.concat([cleaned_df.iloc[:, 0:2], scaled_imputed_df],axis=1)


Merging Data with WRI

In [ ]:
data = pd.read_csv('/content/drive/My Drive/knn_imputed_dataset.csv')
wri = pd.read_csv('/content/drive/My Drive/worldriskindex-trend.csv')

data = data.rename(columns={'Country Code': 'ISO3.Code'})

data['Year'] = data['Time Code'].str.extract(r'(\d{4})').astype(int)  # Extract year from 'YR2000' and convert to integer

wri_selected_columns = wri[['ISO3.Code', 'Year', 'W']]

merged_data = data.merge(wri_selected_columns, on=['ISO3.Code', 'Year'], how='left')

merged_data=merged_data.drop(columns=['Year'])
merged_data=merged_data.rename(columns={'ISO3.Code':'Country Code'})
print("Shape of the merged dataframe:", merged_data.shape)
print("Sample of the merged dataframe:")
print(merged_data.head())

merged_data.to_csv('/content/drive/My Drive/merged_data_with_W.csv', index=True)


Correlation Matrix

In [ ]:
df = merged_data
correlation_matrix = df.iloc[:,3:].corr()

#taking only most correlated values
filtered_corr_matrix = correlation_matrix[(correlation_matrix > 0.5) | (correlation_matrix < -0.5)].replace(np.nan, '')

Principle Components Analysis

In [ ]:
pca = PCA(0.85)
xpca = pca.fit_transform(xtrain)
xpca.shape

# running pca on linear regression
model = LinearRegression()
model.fit(xpca, ytrain)
model.score(xpca, ytrain)
xtest = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest, columns=x.columns)
xpca_test = pca.transform(xtest)
ypred = model.predict(xpca_test)
results = model_test(ytest, ypred)

HDBScan

In [ ]:
file_path = "/content/drive/My Drive/merged_data_with_W.csv"
data = pd.read_csv(file_path)


data['Year'] = data['Time Code'].str.extract('(\d{4})').astype(int)


data = data.sort_values(by=["Country Code", "Year"])

# Select only target numeric columns for clustering
numerical_cols = [
    "Human capital index (HCI) (scale 0-1) [HD.HCI.OVRL]",
    "Current health expenditure per capita, PPP (current international $) [SH.XPD.CHEX.PP.CD]",
    "GNI per capita, PPP (current international $) [NY.GNP.PCAP.PP.CD]",
    "Machinery and transport equipment (% of value added in manufacturing) [NV.MNF.MTRN.ZS.UN]",
    "Individuals using the Internet (% of population) [IT.NET.USER.ZS]",
    "Imports of goods and services (current US$) [NE.IMP.GNFS.CD]",
    "Electric power consumption (kWh per capita) [EG.USE.ELEC.KH.PC]",
    "Power outages in firms in a typical month (number) [IC.ELC.OUTG]",
    "Cost of business start-up procedures (% of GNI per capita) [IC.REG.COST.PC.ZS]",
    "Logistics performance index: Overall (1=low to 5=high) [LP.LPI.OVRL.XQ]"
]

data_numeric = data[numerical_cols].dropna()  # Drop rows with missing values

# Normalize the data for clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_numeric)

clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=5, metric='euclidean')
cluster_labels = clusterer.fit_predict(data_scaled)

data['Cluster'] = cluster_labels

data.to_csv("/content/drive/My Drive/clustered_data.csv", index=False)
print("Clustered data saved to clustered_data.csv")

# PCA for 2D visualization
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)
data['PCA1'] = pca_result[:, 0]
data['PCA2'] = pca_result[:, 1]

plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PCA1', y='PCA2', hue='Cluster', palette='tab10', data=data, legend='full'
)
plt.title("HDBSCAN Clustering (PCA Projection)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Cluster", loc="best")
plt.grid()
plt.show()

# Cluster-wise LPI Analysis
plt.figure(figsize=(12, 6))
sns.boxplot(
    x='Cluster',
    y="Logistics performance index: Overall (1=low to 5=high) [LP.LPI.OVRL.XQ]",
    data=data
)
plt.title("LPI Distribution by Cluster")
plt.xlabel("Cluster")
plt.ylabel("Logistics Performance Index")
plt.grid()
plt.show()

outliers = data[data['Cluster'] == -1]

outlier_info = outliers[['Country Code', 'Year']]

# Display the outlier information
print("Outlier Countries and Years:")
print(outlier_info)

data['Cluster'] = cluster_labels

cluster_0_data = data[data['Cluster'] == 0]

cluster_0_countries = cluster_0_data[['Country Code', 'Year']]

# Display the names and years of countries in cluster "1"
print(cluster_0_countries)


Modelling a Multivariate Elastic Net Regression

In [ ]:
data = scaled_imputed_final_df
lpi_colsnames = [col for col in data if 'Logistics' in col]
lpi_colsnames.append('Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]')

y = data[[lpi_colsnames]]
x = data.iloc[:,1:].drop(lpi_colsnames, axis=1)

xtrain, xtest, ytrain, ytest = train_test_split(x, y1, test_size=0.2, random_state=100)

xtrain_scaled = scaler.fit_transform(xtrain)
ytrain_scaled = scaler.fit_transform(ytrain)

xtest_scaled = scaler.fit_transform(xtest)
ytest_scaled = scaler.fit_transform(ytest)

multitaskelasticnet = MultiTaskElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=5, random_state=42)
multitaskelasticnet.fit(xtrain_scaled, ytrain_scaled)

ypred = multitaskelasticnet.predict(xtest_scaled)
results = test_model(ytest, ypred)

Fitting the model with all the data we have

In [ ]:
x_scaled = scaler.fit_transform(x)
y_scaled = scaler.fit_transform(y)
multitaskelasticnet_fin = MultiTaskElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=5, random_state=100)
multitaskelasticnet_fin.fit(x_scaled, y_scaled)

Performing a SHAP analysis to see how the model predicts data

In [ ]:
explainer = shap.Explainer(multitaskelasticnet_fin.predict, x_scaled)
shap_values = explainer(x_scaled, max_evals=600)
list_of_features = x.columns.to_list()
shap.summary_plot(shap_values[:,:,0], features=x, feature_names=list_of_features)

In [ ]:
shap.waterfall_plot(
    shap.Explanation(values=shap_values[0, :, 0],
                     data=x.iloc[0],
                     feature_names=list_of_features)
)

# Forecasting Model

## Using linear regression to predict each country's economic metrics for the next five years

In [ ]:
y = data[lpi_colsnames]
x = data.iloc[:,1:].drop(lpi_colsnames, axis=1)

feats = data.drop(lpi_colsnames, axis=1)

for year in range(2024,2030):
    new_rows_list = []
    for country in feats['Country Code'].unique():
        new_rows = {'Country Code': country,'Time': year}
        for feature in feature_names:
            temp = feats[feats['Country Code'] == country][['Country Code', 'Time', feature]]
            xval = temp[['Time']]
            yval = temp[[feature]]
            model = LinearRegression()
            model.fit(xval, yval)
            prediction = model.predict([[year]])
            pred_feature = {feature: prediction[0]}
            new_rows.update(pred_feature)
        new_rows_list.append(new_rows)

    new_year_preds = pd.DataFrame(new_rows_list)
    new_year_preds = new_year_preds.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    feats = pd.concat([feats, new_year_preds], axis=0)


Using the new feature matrix with predicted values for 2024-2029 and the fitted Elastic Net model to predict the outcome of LPI and Freight.

In [ ]:
ypred = multitaskelasticnet_fin.predict(feats.iloc[:,1:])
pred_colnames = y.columns.to_list()
ypred_df = pd.DataFrame(ypred, columns = pred_colnames)

feats = feats.reset_index(drop=True)
ypred_df = ypred_df.reset_index(drop=True)

forecasted_data = pd.concat([feats, ypred_df], axis=1)